In [1]:
import asyncio
import pandas as pd
import json
import requests
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage
from sentence_transformers import SentenceTransformer
import faiss
import spacy

## System Components and Workflow
The system consists of four main AI agents, each responsible for a specific step in the API integration process.

- **Search Agent (Finding Relevant APIs)**
Uses FAISS to store and retrieve API documentation efficiently.
Extracts keywords from user queries using SpaCy.
Encodes text using SentenceTransformers for similarity search.
- **Code Agent (Generating API Integration Code)**
Uses Ollama (Llama3) to generate backend (FastAPI) and frontend (JavaScript) code.
Ensures the generated API calls follow Sikka's specifications.
- **Review Agent (Evaluating and Scoring Code Quality)**
Reviews generated code for errors, inefficiencies, and best practices.
Provides a quality score (1-10) to determine if modifications are needed.
- **Manager Agent (Orchestrating Workflow and Decision-Making)**
Decides whether the code should be fixed, retried, or finalized.
Requests further API searches if needed.
Formats the final submission output.
- **Submit Agent (Final Code Formatting)**
Summarizes and structures the final output.
Incorporates feedback from the review process.
Ensures the submission is well-documented and formatted for readability.

In [2]:
nlp = spacy.load("en_core_web_sm") # keyword extraction

In [3]:
# class SearchAgent:
#     """Search API documentation using FAISS and refine results using LLM"""
#     def __init__(self, docs, embedder, index, doc_map, model_url="http://localhost:11434/v1", key="ollama", model="llama3.2:latest"):
#         self.docs = docs
#         self.embedder = embedder
#         self.index = index
#         self.doc_map = doc_map
#         self.llm = OpenAIChatCompletionClient(
#             model=model, base_url=model_url, api_key=key,
#             model_info={"vision": False, "function_calling": True, "json_output": False, "family": "unknown"}
#         )

#     def search(self, query: str, top_k=3):
#         """Retrieve the most relevant API documentation based on user request using FAISS search"""
#         query_vec = self.embedder.encode([query])
#         _, indices = self.index.search(query_vec, top_k)
#         return [self.doc_map[i] for i in indices[0] if i in self.doc_map]

#     async def query_api(self, question, retrieved_docs):
#         """Use LLM to analyze API details and return structured answers"""
#         combined_docs = "\n\n".join(retrieved_docs)
#         messages = [
#             {"role": "system", "content": f"You are an AI assistant helping developers integrate Sikka APIs.\nRelevant API documentation:\n{combined_docs}"},
#             {"role": "user", "content": f"Question: {question}\nAnalyze the above API documentation and provide a structured answer."}
#         ]
#         response = await self.llm.create([UserMessage(content=m["content"], source=m["role"]) for m in messages])
#         return response.content.strip()

In [4]:
import spacy
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage

class SearchAgent:
    """Search API documentation using FAISS and refine results using LLM"""

    def __init__(self, docs, embedder, index, doc_map, model_url="http://localhost:11434/v1", key="ollama", model="llama3.2:latest"):
        self.docs = docs
        self.embedder = embedder
        self.index = index
        self.doc_map = doc_map
        self.nlp = spacy.load("en_core_web_sm")  # Load SpaCy for keyword extraction

        self.llm = OpenAIChatCompletionClient(
            model=model, base_url=model_url, api_key=key,
            model_info={"vision": False, "function_calling": True, "json_output": False, "family": "unknown"}
        )

    def extract_keywords(self, query):
        """Extracts important keywords from the user query to improve search accuracy"""
        doc = self.nlp(query)
        keywords = [token.lemma_ for token in doc if token.pos_ in ["NOUN", "VERB", "PROPN"]]  
        return " ".join(keywords)  

    def search(self, query: str, top_k=3):
        """Retrieves the most relevant API documents using FAISS"""
        refined_query = self.extract_keywords(query)
        print(f"Extracted query keywords: {refined_query}")

        query_vec = self.embedder.encode([refined_query])
        _, indices = self.index.search(query_vec, top_k)

        return [self.doc_map[i] for i in indices[0] if i in self.doc_map]

    async def query_api(self, question, retrieved_docs):
        """Uses LLM to analyze API details and return structured answers"""
        combined_docs = "\n\n".join(retrieved_docs)
        messages = [
            {"role": "system", "content": f"You are an AI assistant helping developers integrate Sikka APIs.\nRelevant API documentation:\n{combined_docs}"},
            {"role": "user", "content": f"Question: {question}\nAnalyze the above API documentation and provide a structured answer."}
        ]
        response = await self.llm.create([UserMessage(content=m["content"], source=m["role"]) for m in messages])
        return response.content.strip()


In [5]:
class CodeAgent:
    """Generate API integration code using relevant API documentation"""
    def __init__(self, model_url="http://localhost:11434/v1", key="ollama", model="llama3.2:latest"):
        self.client = OpenAIChatCompletionClient(
            model=model, base_url=model_url, api_key=key,
            model_info={"vision": False, "function_calling": True, "json_output": False, "family": "unknown"}
        )

    async def chat(self, messages):
        """Send input to the language model and retrieve a response"""
        msg_list = [UserMessage(content=m["content"], source=m["role"]) for m in messages]
        response = await self.client.create(msg_list)
        return response.content.strip()

    async def generate_code(self, request: str, docs):
        """Generate backend and frontend integration code using relevant API documentation"""
        docs_text = "\n\n".join(docs)
        messages = [
            {"role": "system", "content": f"Generate code using Sikka APIs. Relevant documentation:\n{docs_text}"},
            {"role": "user", "content": f"Requirement: {request}\nGenerate backend (FastAPI/Flask) and frontend (JavaScript) code."}
        ]
        return await self.chat(messages)

    async def fix_code(self, request: str, feedback: str, original_code: str):
        """Modify generated code based on ReviewAgent's feedback"""
        messages = [
            {"role": "system", "content": f"Fix the following code based on review feedback:\n{original_code}\n\nFeedback:\n{feedback}"},
            {"role": "user", "content": f"Requirement: {request}\nPlease improve the code while keeping it simple and correct."}
        ]
        return await self.chat(messages)

In [6]:
class ReviewAgent:
    """Review the generated code and provide feedback"""
    def __init__(self, model_url="http://localhost:11434/v1", key="ollama", model="llama3.2:latest"):
        self.llm = OpenAIChatCompletionClient(
            model=model, base_url=model_url, api_key=key,
            model_info={"vision": False, "function_calling": True, "json_output": False, "family": "unknown"}
        )

    async def review(self, code: str):
        """Evaluate whether the generated code follows best practices"""
        messages = [
            {"role": "system", "content": "You are an AI code reviewer. Review the given API integration code and identify issues."},
            {"role": "user", "content": f"Code to review:\n{code}\n\nList any potential issues and improvements."}
        ]
        response = await self.llm.create([UserMessage(content=m["content"], source=m["role"]) for m in messages])
        return response.content.strip()

In [7]:
class SubmitAgent:
    """Summarize and format the final output for submission to the client"""
    def __init__(self, model_url="http://localhost:11434/v1", key="ollama", model="llama3.2:latest"):
        self.llm = OpenAIChatCompletionClient(
            model=model, base_url=model_url, api_key=key,
            model_info={"vision": False, "function_calling": True, "json_output": False, "family": "unknown"}
        )

    async def format_submission(self, code: str, review: str, decision: str):
        """Summarize and refine the final code with structured comments."""
        messages = [
            {"role": "system", "content": "You are an AI assistant that formats code for submission. Improve readability, add necessary comments, and structure the response in a professional format."},
            {"role": "user", "content": f"Here is the final code:\n{code}\n\nReview feedback:\n{review}\n\nManager decision:\n{decision}\n\nFormat this into a clean, well-documented output for submission."}
        ]
        response = await self.llm.create([UserMessage(content=m["content"], source=m["role"]) for m in messages])
        return response.content.strip()

In [8]:
class Manager:
    """Handle API search, code generation, and review process using LLM"""
    def __init__(self, search_agent, code_agent, review_agent, submit_agent, model_url="http://localhost:11434/v1", key="ollama", model="llama3.2:latest"):
        self.search_agent = search_agent
        self.code_agent = code_agent
        self.review_agent = review_agent
        self.submit_agent = submit_agent
        self.llm = OpenAIChatCompletionClient(
            model=model, base_url=model_url, api_key=key,
            model_info={"vision": False, "function_calling": True, "json_output": False, "family": "unknown"}
        )

    async def decide_action(self, user_request, code, feedback):
        """Use LLM to decide whether to fix code, retry search, or return result"""
        messages = [
            {"role": "system", "content": "You are an intelligent AI that manages API integration workflows. Your job is to decide whether the code needs to be fixed or if API search should be refined."},
            {"role": "user", "content": f"User request: {user_request}\n\nGenerated code:\n{code}\n\nReview feedback:\n{feedback}\n\nShould the code be fixed, or should API selection be improved? Provide a clear action plan."}
        ]
        response = await self.llm.create([UserMessage(content=m["content"], source=m["role"]) for m in messages])
        return response.content.strip()

    async def process_request(self, user_request):
        """Find relevant API documentation, generate code, review it, and decide next steps"""
        docs = self.search_agent.search(user_request)
        code = await self.code_agent.generate_code(user_request, docs)
        feedback = await self.review_agent.review(code)

        decision = await self.decide_action(user_request, code, feedback)

        if "fix the code" in decision.lower():
            print("Manager decided to request CodeAgent to modify the code...")
            fixed_code = await self.code_agent.fix_code(user_request, feedback, code)
            final_code = fixed_code
        elif "search again" in decision.lower():
            print("Manager decided to refine API search...")
            refined_docs = self.search_agent.search(user_request + " API integration")
            refined_code = await self.code_agent.generate_code(user_request, refined_docs)
            final_code = refined_code
        else:
            final_code = code

        submission = await self.submit_agent.format_submission(final_code, feedback, decision)
        return submission

### Method1: Suitable for one-time execution or when the API changes frequently.

In [9]:
# async def run_system(user_request):
#     """Initialize agents and process a user request for API integration"""
#     try:
#         api_df = pd.read_csv("Sikka_APIs - Sikka_APIs.csv")
#         docs = [
#             f"API: {row.get('API Name', '')}\nDescription: {row.get('Description', '')}\n"
#             f"Endpoints: {row.get('API Endpoints', '')}\nDoc: {row.get('Document Link', '')}"
#             for _, row in api_df.iterrows()
#         ]
#     except:
#         docs = ["Sikka APIs include payments, patient data, scheduling, billing, and financial services."]

#     embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
#     doc_vectors = embedder.encode(docs)
#     index = faiss.IndexFlatL2(doc_vectors.shape[1])
#     index.add(doc_vectors)
#     doc_map = {i: docs[i] for i in range(len(docs))}
    
#     search_agent = SearchAgent(docs, embedder, index, doc_map)
#     code_agent = CodeAgent()
#     review_agent = ReviewAgent()
#     submit_agent = SubmitAgent()
#     manager = Manager(search_agent, code_agent, review_agent, submit_agent)
    
#     return await manager.process_request(user_request)

### Method2: Suitable for production environments, with reusable indexes.

In [10]:
import os

def load_or_build_faiss_index(docs, embedder, index_path="sikka_api.index"):
    """Load an existing FAISS index, or create and save a new one if not found"""
    if os.path.exists(index_path):
        print("Loading existing FAISS index...")
        index = faiss.read_index(index_path)
    else:
        print("Computing embeddings and creating a new index...")
        doc_vectors = embedder.encode(docs)
        index = faiss.IndexFlatL2(doc_vectors.shape[1])  # Create an L2 distance index
        index.add(np.array(doc_vectors).astype("float32"))
        faiss.write_index(index, index_path)  # Save the index

    return index

async def run_system(user_request):
    """Initialize the agents and process the user request"""
    try:
        api_df = pd.read_csv("Sikka_APIs - Sikka_APIs.csv")
        docs = [
            f"API: {row.get('API Name', '')}\nDescription: {row.get('Description', '')}\n"
            f"Endpoints: {row.get('API Endpoints', '')}\nDoc: {row.get('Document Link', '')}"
            for _, row in api_df.iterrows()
        ]
    except:
        docs = ["Sikka APIs include payments, patient data, scheduling, billing, and financial services."]

    embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    # Use a persistent FAISS index
    index = load_or_build_faiss_index(docs, embedder)

    # Create an API document index mapping
    doc_map = {i: docs[i] for i in range(len(docs))}
    
    # Initialize agents
    search_agent = SearchAgent(docs, embedder, index, doc_map)
    code_agent = CodeAgent()
    review_agent = ReviewAgent()
    submit_agent = SubmitAgent()
    manager = Manager(search_agent, code_agent, review_agent, submit_agent)
    
    return await manager.process_request(user_request)

 

In [11]:
async def main():
    """Execute the full system with a sample request."""
    request = "I want to build a doctor payment system with FastAPI and a JavaScript frontend."
    result = await run_system(request)
    print(result)

In [12]:
await main()

Loading existing FAISS index...
Extracted query keywords: want build doctor payment system fastapi JavaScript frontend
Based on the review feedback and existing code, I will provide an updated version of the `main.py` file that includes improvements in error handling and API key management:

```python
from fastapi import FastAPI, Depends
import requests

class APIKeySecret:
    def __init__(self, api_key, balance_id, payment_plans_api_key):
        self.api_key = api_key
        self.balance_id = balance_id
        self.payment_plans_api_key = payment_plans_api_key

@app.get("/balance")
def get_patient_balance(patient_id: str, apikey_secret=Depends(APIKeySecret)):
    try:
        base_url = f"https://api.sikkasoft.com/v4/patient_balance/{patient_id}"
        headers = {"Authorization": f"Bearer {apikey_secret.api_key}"}
        
        response = requests.get(base_url, headers=headers)
        
        if response.status_code == 200:
            return response.json()
        else:
 